In [25]:
from nilearn import plotting
from nilearn.image import resample_img, mean_img
from nilearn import datasets
import nibabel as nib
from nipype.interfaces import fsl
from nipype.interfaces.ants import N4BiasFieldCorrection, RegistrationSynQuick
from lib.zscore_norm import zscore_normalize
import utils.applyxfm4d


atlas_folder = "G:/StrokeResearch/vascular territory template/"
data_test_folder = 'G:/StrokeResearch/vascular territory template/testcase/'
atl_terr_path = atlas_folder+'mni_vascular_territories.nii.gz'
atl_flair_path = atlas_folder + 'caa_flair_in_mni_template_smooth.nii.gz'

In [20]:
pwi_path = data_test_folder+'540016/PWI.nii.gz'
pwi = nib.load(pwi_path)
print(pwi.shape)

(128, 128, 24, 90)


In [21]:
flair_path = data_test_folder+'540016/FLAIR.nii.gz'
flair = nib.load(flair_path)
print(flair.shape)

(232, 256, 26)


In [23]:
atl_terr = nib.load(atl_terr_path)
atl_flair = nib.load(atl_flair_path)
print(atl_flair.shape)

(182, 218, 182)


In [26]:
## manual registration starts here

reorient = fsl.utils.Reorient2Std()
reorient.inputs.in_file = flair_path
reorient.inputs.out_file = data_test_folder+'FLAIR.nii.gz'
res = reorient.run() 

200917-15:05:34,383 nipype.interface WARNING:
	 FSLOUTPUTTYPE environment variable is not set. Setting FSLOUTPUTTYPE=NIFTI


OSError: No command "fslreorient2std" found on host HarryPC. Please check that the corresponding package is installed.

In [ ]:
# robust fov to remove neck and lower head automatically

rf = fsl.utils.RobustFOV()
rf.inputs.in_file = data_test_folder+'FLAIR.nii.gz'
rf.inputs.out_roi = data_test_folder+'FLAIR_RF.nii.gz'
rf.cmdline
rf.run()

In [ ]:
# two run experiment
btr1 = fsl.BET()
btr1.inputs.in_file = data_test_folder+'FLAIR_RF.nii.gz'
btr1.inputs.robust = True
btr1.inputs.frac = 0.5
btr1.inputs.out_file = data_test_folder+'FLAIR_RF.nii.gz'
res = btr1.run()

In [ ]:
n4 = N4BiasFieldCorrection()
n4.inputs.dimension = 3
n4.inputs.input_image = data_test_folder+'FLAIR_RF.nii.gz'
n4.inputs.bspline_fitting_distance = 300
n4.inputs.shrink_factor = 3
n4.inputs.n_iterations = [50,50,30,20]
n4.inputs.output_image = data_test_folder+'FLAIR_RF.nii.gz'
res = n4.run()

In [ ]:
flt = fsl.FLIRT(bins=640, cost_func='mutualinfo', interp='spline',
                searchr_x=[-180, 180], searchr_y=[-180, 180], searchr_z=[-180,180],dof=12)
flt.inputs.in_file = data_test_folder+'FLAIR_RF.nii.gz'
flt.inputs.reference = atl
flt.inputs.out_file = data_test_folder+'FLAIR_fisrt_run_r.nii.gz'
flt.inputs.out_matrix_file = data_test_folder+'FLAIR_r_transform.nii.gz'
res = flt.run()

In [ ]:
btr2 = fsl.BET()
btr2.inputs.in_file = data_test_folder+'FLAIR_fisrt_run_r.nii.gz'
btr2.inputs.robust = True
btr2.inputs.frac = 0.35
btr2.inputs.mask = True
btr2.inputs.out_file = data_test_folder+'FLAIR_r.nii.gz'
res = btr2.run()

In [ ]:
# z score normalization
FLAIR_path = data_test_folder+'FLAIR_r.nii.gz'
FLAIR_final = nib.load(FLAIR_path)
FLAIR_mask_path = data_test_folder+ 'FLAIR_mask.nii.gz'
mask = nib.load(FLAIR_mask_path)
FLAIR_norm = zscore_normalize(FLAIR_final, mask)
nib.save(FLAIR_norm, FLAIR_path)

In [ ]:
plotting.plot_anat(data_test_folder+'FLAIR.nii.gz')

In [ ]:
# register with pwi

# automatic reorient to MNI direction
reorient = fsl.utils.Reorient2Std()
reorient.inputs.in_file = pwi_path
reorient.inputs.out_file = data_test_folder+'PWI.nii.gz'
res = reorient.run() 
                
applyxfm4d = applyxfm4d.ApplyXfm4D()
applyxfm4d.inputs.in_file = data_test_folder+'PWI.nii.gz'
applyxfm4d.inputs.single_matrix = data_test_folder+'PWI_r_transform.nii.gz'
applyxfm4d.inputs.out_file =  data_test_folder+'PWI.nii.gz'
applyxfm4d.inputs.ref_vol = data_test_folder+'FLAIR.nii.gz'
result = applyxfm4d.run()